In [1]:
%pip install python-dotenv jupysql duckdb duckdb-engine sqlalchemy openai twitter-api-client --quiet

Note: you may need to restart the kernel to use updated packages.


# Elementus API Usage Example

In [1]:
from elementus import ElementusClient, ElementusAPIError
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from helper import *
import os

# Load the environment variables
load_env()

# Retrieve Google BigQuery connection string from the environment
GBQ_URI = get_gbq_uri()
engine = create_engine(GBQ_URI)

# Retrieve the API key from the environment
ELEMENTUS_API_KEY = get_elementus_api_key()

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Retrieve last Bitcoin block transactions over 1 BTC

In [2]:
%load_ext sql
%sql engine

In [3]:
%%sql result <<
WITH max_block AS (
  SELECT MAX(number) as max_number 
  FROM `bigquery-public-data.crypto_bitcoin.blocks`
),
inputs AS (
  SELECT DISTINCT
    transaction_hash AS tx_hash,
    addresses
  FROM `bigquery-public-data.crypto_bitcoin.inputs`, max_block
  WHERE block_number = max_number
),
outputs AS (
  SELECT 
    transaction_hash AS tx_hash,
    addresses[OFFSET(0)] AS address,
    value / 100000000 AS value
  FROM `bigquery-public-data.crypto_bitcoin.outputs`, max_block
  WHERE block_number = max_number
)
SELECT 
    o.tx_hash, 
    o.address AS output_address, 
    o.value,
    ARRAY_AGG(DISTINCT input_address) AS input_addresses
FROM outputs o
JOIN inputs i ON o.tx_hash = i.tx_hash
CROSS JOIN i.addresses AS input_address
WHERE o.address NOT IN UNNEST(i.addresses) AND 
o.address NOT IN ('bc1qv4r4zwvf5ckdx3llaykl0vez4fdaj4jh5yfjsw') AND 
i.addresses[OFFSET(0)] NOT IN ('bc1qv4r4zwvf5ckdx3llaykl0vez4fdaj4jh5yfjsw') AND 
o.value >= 1
GROUP BY o.tx_hash, o.address, o.value
ORDER BY value DESC

Running query in 'bigquery://elementus-prod-242016'

8 rows affected.

In [4]:
df = result.DataFrame()
df

,tx_hash,output_address,value,input_addresses
0,cff638019e1852e30fe5a69a540ce674dc7483c5159364...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,499.21471981,[12V12HjcpwCwiMutxdfn3QP76oqp9gjZRh]
1,f642199c3ee74d99b4584cb0b6e99a6aa506039da847ad...,1GrwDkr33gT6LuumniYjKEGjTLhsL5kmqC,84.47186308,"[1M59EwvjrSSmwqKbyWUif7ZsNZGb3S6CNN, 1N4aTCWCi..."
2,d091a37f2ab54e84ab1296aa254ee961c37770b2894919...,bc1qng0keqn7cq6p8qdt4rjnzdxrygnzq7nd0pju8q,2.22540121,"[3CBjVk4CufWfuaYACxEzVfy5yRPY6fqrvn, bc1qh6tlt..."
3,2fb1c310145ec6b36f4acb28642aa98a0fb06bf9bf0cc2...,bc1qx3ytjsdf5zmu2ls9gc5p0ugnflaelpqq84zq44,2.19902378,"[38NmTRpLrjrDKMZLXZNHYa3urgCKBNFsfd, 3FGC9CNFe..."
4,8df8416fc8701b59701bd0aae1748e9e4aa927a8ae733d...,bc1qs048ewtkyxystsnz6t4f7hrmrjzva3m24gxjzq,1.99033063,[bc1q4npt32duldtghdmk5cxvhuzxf3fn44r5jvdqvz]
5,d361e1c75e66d2aae8212066fbbbff9cb4c163337ebf49...,1GrwDkr33gT6LuumniYjKEGjTLhsL5kmqC,1.86681311,"[1Gat2Se8mi384E8C57SRouCoTKsfYXcNJ, 1EppXDPdt1..."
6,bc60ec6eabdc11bf6138167c2ffba7e12ebada62ca5a78...,bc1q9gaphdcrqzkz6aymnwd6mvprm24zkkhzlrksrt,1.66256266,"[38AnUmM95LJGBoj8iB9JHonXLBBZaXy2bm, 33CgSPQiz..."
7,6fd49ddaa659c6a7de565429eb4ed1f9fdcbcd57043948...,bc1qf5alfy4s3qs0dqgrqgn29pgqr55cfk9fn5kcm3,1.35486199,[bc1qg5zl4h4k9ar4t9w8tazxwv8ks0ylyc4rj64mun]


### Use Elementus API to retrieve entity attributions for input and output addresses

In [5]:
client = ElementusClient(api_key=ELEMENTUS_API_KEY)

# Check API health
is_healthy = await client.check_health()

output_addresses = df['output_address'].unique().tolist()
input_addresses = list(set(addr for sublist in df['input_addresses'] for addr in sublist))

try:
    response = await client.get_address_attributions(output_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

# Create a dictionary from the response data
attribution_dict = {address: attr.entity for address, attr in response.data.items()}

try:
    response = await client.get_address_attributions(input_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

attribution_dict.update({address: attr.entity for address, attr in response.data.items()})

len(attribution_dict)

347

### Add entity attributions to transaction data

In [6]:
df['output_entity'] = df['output_address'].map(attribution_dict)
df['input_entity'] = [
    next((attribution_dict[key] for key in addresses if key in attribution_dict), None)
    for addresses in df['input_addresses']
]

df.dropna(subset=['input_entity', 'output_entity'], how='all', inplace=True)
df = df[df['input_entity'] != df['output_entity']]
# Ensure 'input_entity' is a string type before using str.startswith
df = df[~df['input_entity'].astype(str).str.startswith('fastmoney_') & ~df['output_entity'].astype(str).str.startswith('fastmoney_')]

df

,tx_hash,output_address,value,input_addresses,output_entity,input_entity
0,cff638019e1852e30fe5a69a540ce674dc7483c5159364...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,499.21471981,[12V12HjcpwCwiMutxdfn3QP76oqp9gjZRh],coinbase_prime,None
3,2fb1c310145ec6b36f4acb28642aa98a0fb06bf9bf0cc2...,bc1qx3ytjsdf5zmu2ls9gc5p0ugnflaelpqq84zq44,2.19902378,"[38NmTRpLrjrDKMZLXZNHYa3urgCKBNFsfd, 3FGC9CNFe...",NaN,coinbase.com
6,bc60ec6eabdc11bf6138167c2ffba7e12ebada62ca5a78...,bc1q9gaphdcrqzkz6aymnwd6mvprm24zkkhzlrksrt,1.66256266,"[38AnUmM95LJGBoj8iB9JHonXLBBZaXy2bm, 33CgSPQiz...",NaN,coinbase.com


In [7]:
# Convert DataFrame to JSON string
json_string = df[['tx_hash', 'input_entity', 'output_entity', 'value']].to_json(orient='records')

json_string

'[{"tx_hash":"cff638019e1852e30fe5a69a540ce674dc7483c5159364e673d7afe32302377e","input_entity":null,"output_entity":"coinbase_prime","value":499.21471981},{"tx_hash":"2fb1c310145ec6b36f4acb28642aa98a0fb06bf9bf0cc26f7717b0b98501a352","input_entity":"coinbase.com","output_entity":null,"value":2.19902378},{"tx_hash":"bc60ec6eabdc11bf6138167c2ffba7e12ebada62ca5a787c435aba506b1fd421","input_entity":"coinbase.com","output_entity":null,"value":1.66256266}]'

### Create X Post using X AI to generate on-chain insights from transaction data

In [12]:
from openai import OpenAI
from IPython.display import display, Markdown

#XAI_API_KEY = get_xai_api_key()
OPENAI_API_KEY = get_openai_api_key()

client = OpenAI(
    api_key=OPENAI_API_KEY,
    #base_url="https://api.x.ai/v1",
    base_url="https://api.openai.com/v1",
)

completion = client.chat.completions.create(
    #model="grok-2-latest",
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are an expert in Bitcoin blockchain analytics and market structure"
        },
        {
            "role": "user",
            "content": "Given a list of Bitcoin transactions attributed to entities and news currently trending on Crypto Twitter create an X post explaining the most inportant on-chain insights, which may affect BTC/USD price. Provide only the text ready to be sent via X API without any additional comments. Limit the text to 160 characters. " + json_string
        },
    ],
    temperature=0.8,
)

display(Markdown(f"## X Post\n{completion.choices[0].message.content}"))


## X Post
Large BTC moves from wintermute.com & crypto.com to exchanges could signal potential price changes. Stay alert for volatility. #Bitcoin #CryptoNews

### Tweet the generated post

In [10]:
from twitter.account import Account

account = Account(cookies={"ct0": "", "auth_token": ""})
account.tweet(text=completion.choices[0].message.content)

{'data': {'create_tweet': {'tweet_results': {'result': {'rest_id': '1890860377707073623',
     'has_birdwatch_notes': False,
     'core': {'user_results': {'result': {'__typename': 'User',
        'id': 'VXNlcjo5MDA3MjgwNTgyNTYxMzgyNDA=',
        'rest_id': '900728058256138240',
        'affiliates_highlighted_label': {},
        'has_graduated_access': True,
        'is_blue_verified': True,
        'profile_image_shape': 'Circle',
        'legacy': {'can_dm': True,
         'can_media_tag': True,
         'created_at': 'Thu Aug 24 14:34:38 +0000 2017',
         'default_profile': False,
         'default_profile_image': False,
         'description': 'We enable companies of all sizes to make better, faster decisions on the blockchain',
         'entities': {'description': {'urls': []},
          'url': {'urls': [{'display_url': 'elementus.io',
             'expanded_url': 'https://elementus.io/',
             'url': 'https://t.co/h7wrEp5WjF',
             'indices': [0, 23]}]}},
    